In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None) 

import datetime as dt
import time

from calitp import get_engine
from calitp.tables import tbls

engine = get_engine()
connection = engine.connect()

GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/ahsc_grant/'

import gcsfs
fs = gcsfs.GCSFileSystem()

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


# Collating Big Stop Table

In [2]:
# read in and concatenate all geoparquets
fs_list = fs.ls(f"{GCS_FILE_PATH}tool_data/")

#fs_list[1:]

In [3]:
filelist = []
for f in fs_list[1:]:
    try: 
        test_gpqt = gpd.read_parquet(f"gs://{f}")
        filelist.append(f)
    except:
        print(f"error on {f.split('tool_data/')}")
        
#filelist

error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_116.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_118.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_186.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_199.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_21.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_212.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_213.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_216.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_23.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_254.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_277.parquet']
error on ['c

In [10]:
gdf_wkd = (pd.concat(
        gpd.read_parquet(f"gs://{f}")
        for f in filelist
        )
           >> filter(_.daytype=="Weekday")
          )

In [11]:
gdf_wkd.head()

,calitp_itp_id,stop_name,stop_id,geometry,n_trips,n_routes,daytype,analysis_date,sum_tracts,sum_total_pop,sum_households,sum_not_us_citizen_pop,sum_youth_pop,sum_seniors_pop,sum_pop_determined_poverty_status,sum_poverty,sum_no_car,sum_no_cars,sum_land_area,sum_jobs,land_area_sqkm,pop_density,job_density,pct_not_us_citizen_pop,pct_youth_pop,pct_seniors_pop,pct_poverty,pct_pop_workers_no_car,pct_hh_no_cars
0,10,Stockton Station,3400001,POINT (-121.27895 37.95706),8,0,Weekday,2022-11-02,3,16656.0,6416.0,4894.0,6072.0,1520.0,16456.0,7588.0,804.0,2460.0,7572848.0,36500.0,7.572848,2199.436724,4819.851131,29.382805,36.455331,9.125841,46.110841,4.827089,38.341646
1,10,Stockton Station,3400001,POINT (-121.27895 37.95706),8,0,Weekday,2022-11-02,3,16656.0,6416.0,4894.0,6072.0,1520.0,16456.0,7588.0,804.0,2460.0,7572848.0,36500.0,7.572848,2199.436724,4819.851131,29.382805,36.455331,9.125841,46.110841,4.827089,38.341646
2,10,Lathrop/Manteca Station,3400002,POINT (-121.26366 37.79791),8,0,Weekday,2022-11-02,2,30612.0,8320.0,4548.0,11340.0,2630.0,30588.0,2742.0,54.0,92.0,94456838.0,24000.0,94.456838,324.084531,254.084305,14.856919,37.044296,8.591402,8.964300,0.176401,1.105769
3,10,Lathrop/Manteca Station,3400002,POINT (-121.26366 37.79791),8,0,Weekday,2022-11-02,2,30612.0,8320.0,4548.0,11340.0,2630.0,30588.0,2742.0,54.0,92.0,94456838.0,24000.0,94.456838,324.084531,254.084305,14.856919,37.044296,8.591402,8.964300,0.176401,1.105769
4,10,Tracy Station,3400003,POINT (-121.43387 37.69647),8,0,Weekday,2022-11-02,3,42968.0,11960.0,3100.0,17280.0,2986.0,42884.0,2024.0,340.0,194.0,800367410.0,6538.0,800.367410,53.685344,8.168748,7.214671,40.215975,6.949358,4.719709,0.791287,1.622074


In [16]:
gdf_wkd.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 103376 entries, 0 to 111
Data columns (total 29 columns):
 #   Column                             Non-Null Count   Dtype   
---  ------                             --------------   -----   
 0   calitp_itp_id                      103376 non-null  int64   
 1   stop_name                          103376 non-null  object  
 2   stop_id                            103376 non-null  object  
 3   geometry                           103376 non-null  geometry
 4   n_trips                            103376 non-null  int64   
 5   n_routes                           103376 non-null  int64   
 6   daytype                            103376 non-null  object  
 7   analysis_date                      103376 non-null  object  
 8   sum_tracts                         103376 non-null  int64   
 9   sum_total_pop                      103376 non-null  float64 
 10  sum_households                     103376 non-null  float64 
 11  sum_not_us_citizen_po

In [19]:
gdf_wkd.n_routes.value_counts()

#~2000 stops with no associated routes; missing route_id?

1     66212
2     21511
3      5441
4      4367
0      1949
6      1696
5       722
8       709
7       213
10      209
9       184
12       55
14       52
13       14
11       13
18        8
20        6
16        4
24        3
17        2
19        2
23        2
15        2
Name: n_routes, dtype: int64

Note: Running into memory issues adding spatial weights matrix. Proceeding without spatially-lagged factors for now.

In [5]:
# put coefficients into arrays - variable order from spatial_regression_exploration_kmk
# in future, save out coefficients somewhere 
import numpy as np

wkd_coeff = np.array([-0.1610594,0.0001214,-0.0000173,0.0224169,-0.0152673,-0.0505976,-0.0423512,0.0111763])
sat_coeff = np.array([-0.1424400,0.0001344,-0.0000186,0.0256008,-0.0169793,-0.0408743,-0.0419725,0.0126354])
sun_coeff = np.array([-0.1082477,0.0001477,-0.0000202,0.0209053,-0.0145447,-0.0449611,-0.0502937, 0.0132250])

In [9]:
# multiply by coefficients - everything except n_trips
gdf_wkd.loc[:, ['n_trips', 'n_routes', 'pop_density', 'job_density','pct_not_us_citizen_pop',
                    'pct_youth_pop', 'pct_seniors_pop', 'pct_pop_workers_no_car', 'pct_poverty']] *= wkd_coeff

In [33]:
import numpy as np

In [29]:
# create stop-specific ridership factor
gdf_wkd = (gdf_wkd
           >> mutate(control_vars_sum = _.n_routes+_.pop_density+_.job_density+_.pct_not_us_citizen_pop+_.pct_youth_pop+_.pct_seniors_pop+_.pct_pop_workers_no_car+_.pct_poverty,
                     control_vars_factor = _.control_vars_sum+(_.n_trips*0.0200865)+6.8721538
           )
          )

In [34]:
# baseline ridership model estimate: np.exp(control_vars_factor + (n_trips*n_trips_coeff))
gdf_wkd['model_est_ridership'] = np.exp(gdf_wkd['control_vars_factor'])

/opt/conda/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: overflow encountered in exp


In [35]:
gdf_wkd.model_est_ridership.value_counts()

inf              91570
7.383353e+70        68
8.112785e+71        48
5.942158e+34        44
1.450940e+192       42
                 ...  
3.814801e+66         1
7.444950e+37         1
5.040501e+156        1
5.095598e+34         1
2.413972e+33         1
Name: model_est_ridership, Length: 3800, dtype: int64

In [36]:
gdf_wkd.model_est_ridership.min()

2.463786278729821e+27